In [1]:
import SimpleITK as sitk
from glob import glob
from tqdm import tqdm
import torch
import numpy as np
import os

In [2]:
for patient in tqdm(glob("/home/johannes/Data/SSD_2.0TB/GNN_pCR/data/breast_cancer/data_processed/ISPY2*")):
    # print(patient) 
    # break

    dce_files = sorted(glob(f"{patient}/*DCE_timepoint*resampled*.nii.gz"))
    ftv_files = sorted(glob(f"{patient}/*Analysis*resampled*.nii.gz"))

    for i in range(len(dce_files)//3):
        # print(dce_files[i*3], dce_files[i*3+1], dce_files[i*3+2], ftv_files[i])

        dce_tp0 = sitk.ReadImage(dce_files[i*3])
        dce_tp1 = sitk.ReadImage(dce_files[i*3+1])
        dce_tp2 = sitk.ReadImage(dce_files[i*3+2])
        ftv = sitk.ReadImage(ftv_files[i], sitk.sitkUInt8)

        # Create 3-channel volume
        array_tp0 = sitk.GetArrayFromImage(dce_tp0)
        array_tp1 = sitk.GetArrayFromImage(dce_tp1)
        array_tp2 = sitk.GetArrayFromImage(dce_tp2)
        array_ftv = sitk.GetArrayFromImage(ftv)

        channel_0 = array_tp1 - array_tp0
        channel_1 = array_tp2 - array_tp0
        channel_2 = array_ftv

        array_3ch = np.stack([channel_0, channel_1, channel_2], axis=0)  # Shape: (3, D, H, W)
        torch_3ch = torch.from_numpy(array_3ch).float()
        torch_3ch = torch_3ch.unsqueeze(0)  # Shape: (1, 3, D, H, W)

        patient_id = os.path.basename(dce_files[i*3]).split("_")[0]
        aquistion_time = os.path.basename(dce_files[i*3]).split("_")[1]
        dir_path = os.path.dirname(dce_files[i*3])
        filepath = os.path.join(dir_path, f"{patient_id}_{aquistion_time}.pt")

        torch.save(torch_3ch, filepath)

100%|██████████| 625/625 [00:48<00:00, 12.92it/s]
